Get list of well-developed players and rookies/players with less than 1 year of service time.

In [ ]:
pip install pybaseball -q

Split players into their position (P, C, 1/2/3B, OF, DH)

In [2]:
from pybaseball import playerid_lookup, chadwick_register, statcast_batter, statcast_pitcher
import pandas as pd
import json

# Updated code to account for the 2024 season once it begins.
# df = chadwick_register(save=True)
# df = df.query('mlb_played_first > 2013 and mlb_played_last == 2024')

# relevant_columns = ["name_first", "name_last", "key_mlbam", "mlb_played_first", "mlb_played_last"]

# df = pd.read_pickle('./playerslastdecade.pkl')[relevant_columns].sort_values(by=['name_first']).reset_index(drop=True)

# filtered_player_arr = df.to_json(orient="records")
# with open("playerslastdecade.json", "w") as out:
#     out.write(filtered_player_arr)

Use a prediction model taking into account basic stats plotted against the player's actual value. Play around with which stats to prioritize.

In [9]:
from pybaseball import batting_stats_bref, pitching_stats_bref
import json

dataB = batting_stats_bref(2023)
dataP = pitching_stats_bref(2023)

filtered_data = json.load(open("../sorarehelpers/u_scores_and_ids.json"))
nw_df_arr = [list() for _ in range(6)]

print("BEGIN FILTERING...")

for idx, val in enumerate(filtered_data):
    print("ON TASK", idx + 1, "OF", len(filtered_data))
    if idx < 4:
        for player, obj in val.items():
            # Age, GP, PA, BA, OBP, SLG, R, RBI, HR, SB, SORARE AVG SCORE // PRICE
            df = dataB.query("mlbID == %s" % (obj[1]))
            df.dropna(inplace=True)
            if df.size:
                player_elem = [player, float(df["Age"].iloc[0]), int(df["G"].iloc[0]), int(df["PA"].iloc[0]), float(df["BA"].iloc[0]), float(df["OBP"].iloc[0]), float(df["SLG"].iloc[0]), int(df["R"].iloc[0]), int(df["RBI"].iloc[0]), int(df["HR"].iloc[0]), int(df["SB"].iloc[0]), obj[0], obj[2]]
                nw_df_arr[idx].append(player_elem)
    else:
        if idx == 4:
            for player, obj in val.items():
                # Age, G, W, L, IP, SO, ERA, WHIP, SO9, SORARE AVG SCORE // PRICE (starter)
                df = dataP.query("mlbID == '%s'" % (obj[1]))
                df.dropna(inplace=True)
                if df.size:
                    player_elem = [player, int(df["Age"].iloc[0]), int(df["G"].iloc[0]), int(df["W"].iloc[0]), int(df["L"].iloc[0]), int(df["IP"].iloc[0]), int(df["SO"].iloc[0]), float(df["ERA"].iloc[0]), float(df["WHIP"].iloc[0]), float(df["SO9"].iloc[0]), obj[0], obj[2]]
                    nw_df_arr[idx].append(player_elem)
        else:
            for player, obj in val.items():
                # Age, SV, IP, SO, ERA, WHIP, SO9, SORARE AVG SCORE // PRICE (closer/reliever)
                df = dataP.query("mlbID == '%s'" % (obj[1]))
                df.dropna(inplace=True)
                if df.size:
                    player_elem = [player, int(df["Age"].iloc[0]), int(df["SV"].iloc[0]), int(df["IP"].iloc[0]), int(df["SO"].iloc[0]), float(df["ERA"].iloc[0]), float(df["WHIP"].iloc[0]), float(df["SO9"].iloc[0]), obj[0], obj[2]]
                    nw_df_arr[idx].append(player_elem)

nw_df_arr_json = json.dumps(nw_df_arr, indent=4)
with open("./dfs.json", "w") as out:
    out.write(nw_df_arr_json)

print("DONE!")

BEGIN FILTERING...
ON TASK 1 OF 6


/var/folders/1x/rxyxlt2x5vq06s0v666ymf2w0000gn/T/ipykernel_76939/2375310987.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


ON TASK 2 OF 6
ON TASK 3 OF 6
ON TASK 4 OF 6
ON TASK 5 OF 6


/var/folders/1x/rxyxlt2x5vq06s0v666ymf2w0000gn/T/ipykernel_76939/2375310987.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


ON TASK 6 OF 6


/var/folders/1x/rxyxlt2x5vq06s0v666ymf2w0000gn/T/ipykernel_76939/2375310987.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


DONE!


In [20]:
import pandas as pd
import json
of_df = pd.DataFrame(json.load(open("./dfs.json"))[0], columns="Name, Age, GP, PA, BA, OBP, SLG, R, RBI, HR, SB, SR Score, Price".split(", "))
of_df

# create x (features matrix)
X = of_df.drop("Price", axis=1)
# create y (labels matrix)
y = of_df["Price"]

import sklearn
from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LinearRegresion, Lasso, ElasticNet
from sklearn.svm import LinearSVR, SVR
from sklearn.neural_network import MLPRegressor
clf = MLPRegressor()

from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer

# categorical_features = ["Name"]
# one_hot = OneHotEncoder()
# transformer = ColumnTransformer([("one_hot", one_hot, categorical_features)], remainder="passthrough")
# transformed_X = transformer.fit_transform(X)

dX = X.drop("Name", axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(dX, y, test_size=0.2)

clf.fit(X_train, Y_train)

/opt/homebrew/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor()

In [21]:
clf.score(X_test, Y_test)

0.7791932299601056

In [25]:
df = dataB.query("Name == 'Aaron Judge'")
player_elem = [[float(df["Age"].iloc[0]), int(df["G"].iloc[0]), int(df["PA"].iloc[0]), float(df["BA"].iloc[0]), float(df["OBP"].iloc[0]), float(df["SLG"].iloc[0]), int(df["R"].iloc[0]), int(df["RBI"].iloc[0]), int(df["HR"].iloc[0]), int(df["SB"].iloc[0]), 10]]
temp = pd.DataFrame(player_elem, columns="Age, GP, PA, BA, OBP, SLG, R, RBI, HR, SB, SR Score".split(", "))
clf.predict(temp)

array([42.03843858])